In [1]:
import os
os.environ['HF_HUB_ENABLE_HF_TRANSFER'] = '0'
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import torch
import gc

torch.cuda.empty_cache()
gc.collect()

43

In [3]:
def is_bfloat16_supported():
    """
    Checks if the current device supports bfloat16.
    """
    return torch.cuda.is_available() and torch.cuda.get_device_capability(0)[0] >= 8

In [4]:
from datasets import load_dataset, Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    TrainerCallback,
    PreTrainedTokenizerBase,
)
from trl import GRPOConfig, GRPOTrainer
from peft import LoraConfig, PeftConfig
import re
from vllm import LLM, SamplingParams

INFO 02-12 19:12:30 __init__.py:190] Automatically detected platform cuda.


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [6]:
# meta-llama/Llama-3.2-3B
# HuggingFaceTB/SmolLM2-1.7B-Instruct

In [7]:
# Model configuration.
model_name = "HuggingFaceTB/SmolLM2-1.7B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# Load the pre-trained model on CPU first, then move to GPU.
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    attn_implementation="sdpa",
    quantization_config=bnb_config,
    device_map="auto",
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

In [8]:
# Load and prep dataset
SYSTEM_PROMPT = """
<reasoning>
</reasoning>
<answer>
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

def get_gsm8k_questions(split = "train") -> Dataset:
    data = load_dataset('openai/gsm8k', 'main')[split]
    data = data.map(lambda x: {
        'prompt': [
            {'role': 'user', 'content': SYSTEM_PROMPT + "\n" + x['question']},
        ],
        'answer': extract_hash_answer(x['answer'])
    })
    return data

dataset = get_gsm8k_questions()

scaling_down = 3.

# Reward functions
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content'].split("\n")[-1]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    if hash(q) % 10 == 0:
        pass
        #print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 / scaling_down if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 / scaling_down if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 / scaling_down if match else 0.0 for match in matches]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count / scaling_down

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

In [11]:
peft_config = LoraConfig(
    lora_alpha=64,
    lora_dropout=0,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
)

training_args = GRPOConfig(
    use_vllm=True,  # use vLLM for fast inference!
    vllm_device="cuda:0",
    vllm_gpu_memory_utilization=0.6,
    learning_rate=5e-6,
    adam_beta1=0.9,
    adam_beta2=0.99,
    weight_decay=0.1,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    optim="paged_adamw_8bit",
    bf16=is_bfloat16_supported(),
    fp16=not is_bfloat16_supported(),
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,  # Increase to 4 for smoother training
    num_generations=4,  # Decrease if out of memory
    max_prompt_length=512,
    max_completion_length=256,
    num_train_epochs=1,  # Set to 1 for a full training run
    max_steps=100,
    logging_steps=10,
    save_steps=500,
    max_grad_norm=0.1,
    report_to="tensorboard",
    logging_dir="logs/runs",
    output_dir="outputs",
)

trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args=training_args,
    train_dataset=dataset,
    peft_config=peft_config,
)

# trainer.add_callback(EvalCallback(model, tokenizer, eval_data, device))

trainer.train()

/home/lmassaron/mycode/r1-reasoning/.venv/lib/python3.10/site-packages/trl/trainer/grpo_trainer.py:357: UserWarning: The requested device cuda:0 is also used for training. This may lead to unexpected behavior. It is recommended to use a dedicated device for vLLM.
  warnings.warn(


INFO 02-12 19:13:40 config.py:542] This model supports multiple tasks: {'embed', 'reward', 'classify', 'generate', 'score'}. Defaulting to 'generate'.
INFO 02-12 19:13:40 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='HuggingFaceTB/SmolLM2-1.7B-Instruct', speculative_config=None, tokenizer='HuggingFaceTB/SmolLM2-1.7B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda:0, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=HuggingFaceTB/SmolLM2-1

[W212 19:13:41.526953810 CUDAAllocatorConfig.h:28] Warning: expandable_segments not supported on this platform (function operator())


INFO 02-12 19:13:41 weight_utils.py:252] Using model weights format ['*.safetensors']
INFO 02-12 19:13:42 weight_utils.py:297] No model.safetensors.index.json found in remote.


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 02-12 19:13:42 model_runner.py:1115] Loading model weights took 3.1887 GB
INFO 02-12 19:13:43 worker.py:267] Memory profiling takes 0.71 seconds
INFO 02-12 19:13:43 worker.py:267] the current vLLM instance can use total_gpu_memory (23.68GiB) x gpu_memory_utilization (0.60) = 14.21GiB
INFO 02-12 19:13:43 worker.py:267] model weights take 3.19GiB; non_torch_memory takes 0.02GiB; PyTorch activation peak memory takes 0.48GiB; the rest of the memory reserved for KV Cache is 10.52GiB.
INFO 02-12 19:13:43 executor_base.py:110] # CUDA blocks: 3591, # CPU blocks: 1365
INFO 02-12 19:13:43 executor_base.py:115] Maximum concurrency for 8192 tokens per request: 7.01x
INFO 02-12 19:13:46 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_util

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:13<00:00,  2.60it/s]

INFO 02-12 19:13:59 model_runner.py:1562] Graph capturing finished in 13 secs, took 0.16 GiB
INFO 02-12 19:13:59 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 16.90 seconds


Step,Training Loss


ValueError: There is no module or parameter named 'base_model' in LlamaForCausalLM